# Sparse Systems and Iterative Methods

We'll look at `scipy.sparse` along with some basic principles of iterative methods, which are at the heart of many production-level simulation tools (e.g., most finite-element modelers).

## Required Preparation


- Review [linear systems](http://robertsj.github.io/python_for_engineers/courses/pythonic_apps_2/modules/module_2/module_2.html)


## Project 1

 - Teams of 2 or 3 
 - Problem must be central to a core ME class (e.g., thermo, fluids, heat transfer, machine design)
 - Solution must incorporate something from SciPy, collaborate version control, and unit tests.  *Can* include object-oriented programming and GUIs.
 - Half-page (max) proposal due by 10/11.
 - Final project with three-page (max) report due by 11/08.
 
 
Goal here is on the *development practice* and not on how challenging the problem is, how complicated the solution is, etc.  Breadth and depth will be checked at proposal stage.

# Why Sparse Systems?

Consider our model problem $-k y'' = q$ subject to $y(0) = y_L$ and $y(1) = y_R$.  Application of the central difference led to

$$
 \left (  \begin{matrix}
  1 & 0 & \cdots & &\\
 -\frac{k}{\Delta^2} & \frac{2k}{\Delta^2} & -\frac{k}{\Delta^2} & 0 & \ldots \\
 0 & -\frac{k}{\Delta^2} & \frac{2k}{\Delta^2} & -\frac{k}{\Delta^2} & 0 & \ldots \\
  &   & \ddots &   & \ddots \\
  &   &  & \ldots & 0 & 1\\
 \end{matrix} \right )
 \left (  \begin{matrix}
  y_0 \\
  y_1 \\
  y_2 \\
  \vdots \\
  y_{n-1} \\
 \end{matrix} \right ) 
= 
 \left (  \begin{matrix}
  y_L \\
  q_1 \\
  q_2 \\
  \vdots \\
  y_R \\
 \end{matrix} \right ) 
$$

Questions:

  - How many elements of the matrix are nonzero (assume $k > 0$)?
  - If we use a regular `np.array` to store this matrix, how much does a matrix-vector product cost to compute (i.e., how many floating-point operations does it require)?  
  - Approximately what fraction of those operations are wasted?
  - Could you solve this system on your machine if $n = 10000$?

Let's construct the model system (and solve it) using tools we already have:

In [ ]:
import numpy as np
def make_A_and_b(n, k=0.2, Delta = 0.1, y_L=0, y_R=0, q=1):
    """Make model """
    A = np.zeros((n, n))
    b = np.ones(n)*q
    b[0] = y_L; b[-1] = y_R
    A[0, 0] = 1.0; A[-1, -1] = 1.0
    for i in range(1, n-1):
        A[i, i] = 2*k/Delta**2; A[i, i-1] = -k/Delta**2; A[i, i+1] = -k/Delta**2
    return A, b
A, b = make_A_and_b(5, Delta=1.0, y_L=1)
np.linalg.solve(A, b)

Now, let's explore `scipy.sparse`.  Key commands:

  - `scipy.sparse.csr_matrix`
  - `scipy.sparse.linalg.spsolve`
  - `scipy.sparse.csr_matrix`
  - `scipy.sparse.csr_matrix.todense`
  - `scipy.sparse.csr_matrix.diagonal`
  - `scipy.sparse.diags`
  - `scipy.sparse.tril` and `triu`

In [ ]:
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

In [ ]:
# Create a sparse matrix from A

In [ ]:
# Solve the sparse system

But we don't want to build a dense matrix in the first place...

In [ ]:
# Look at the structure of A_sparse

In [ ]:
# Build A_sparse again from that structure (other ways, too)

## Iterative Methods

Consider $ax = b$.  Easy to solve, but pretend you don't have division.  How about

$$
  x^{new} = (1 - a)x^{old} + b
$$

In [ ]:
a = 0.5
b = 1.0 # expect
x = 0.0
for i in range(100):
    x = (1-a)*x + b
    print("{:.4e} {:.4e}".format(x, x-b/a))

For matrices, this becomes Richardson iteration:

$$
  \mathbf{x}^{new} = (\mathbf{I}-\mathbf{A})\mathbf{x}^{old} + \mathbf{b} \, .
$$

Works only if $||\mathbf{I}-\mathbf{A}|| < 1$, which is not true for many systems, including our model problem!

Ultimately, iteration is good when (1) matrices are *sparse* and (2) the number of iterations is small.

Better option: *precondition* the system with some matrix $\mathbf{P}$, i.e., solve

$$
  \mathbf{P}^{-1} \mathbf{A} \mathbf{x} = \mathbf{P}^{-1} \mathbf{b}
$$

What's the "best" possible $\mathbf{P}$?

## The Classical Schemes

Let $\mathbf{P}$ be the main diagonal $\mathbf{D}$ of $\mathbf{A}$.  Then,

$$
  \mathbf{x}^{new} = (\mathbf{I}-\mathbf{D}^{-1}\mathbf{A})\mathbf{x}^{old} + \mathbf{D}^{-1}\mathbf{b} \, .
$$

This is **Jacobi iteration**.

Let $\mathbf{P}$ be the main diagonal $\mathbf{D+L}$ of $\mathbf{A}$ (i.e., everything on and below the main diagonal.  Then,

$$
  \mathbf{x}^{new} = (\mathbf{I}-\mathbf{(D+U)}^{-1}\mathbf{A})\mathbf{x}^{old} + \mathbf{(D+U)}^{-1}\mathbf{b} \, .
$$

This is **Gauss-Seidel iteration**.

## Advanced Schemes

Several schemes exist that outperform Jacobi and Gauss Seidel. Many of these are *Krylov subspace* methods, e.g., GMRES.  All seek a solution $\mathbf{x} \in \mathcal{K}_n = [\mathbf{x}_0, \mathbf{Ax}_0, \mathbf{A}^2\mathbf{x}_0, \ldots \mathbf{A}^{n-1}\mathbf{x}_0]$.  